# Transfer Learning Models
***
### Additional modeling process using pretrained models.  


Import neccesary packages and libraries 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os, shutil
from keras import models
from keras import layers
from sklearn.metrics import confusion_matrix, f1_score
np.random.seed(123)
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

### Read in Data
Read in data with ImageDataGenerator.  Created validation set from training dataset.  

In [3]:
test_directory = 'Alzheimer_s Dataset/test/'
train_directory = 'Alzheimer_s Dataset/train/'

data_genorator = ImageDataGenerator(rescale=1./255,validation_split=0.2)


data_train= data_genorator.flow_from_directory( 
        train_directory, 
        subset= 'training',
        target_size=(224, 224), 
        batch_size = 4098, 
        seed = 123)

data_valid = data_genorator.flow_from_directory( 
        train_directory, 
        subset= 'validation',
        target_size=(224, 224), 
        batch_size = 1023, 
        seed = 123)

data_test= ImageDataGenerator(rescale=1./255).flow_from_directory( 
        test_directory, 
        subset= 'training',
        target_size=(224, 224), 
        batch_size = 1279, 
        seed = 123)

### Seperate lables from images

In [9]:
train_images, train_labels = next(data_train)

test_images, test_labels = next(data_test)

valid_images, valid_labels = next(data_valid)

In [10]:
def get_label(array):
    if array[0] ==1:
        return 0
    elif array[1] ==1:
        return 1
    elif array[2] ==1:
        return 2
    elif array[3] ==1:
        return 3

In [11]:
train_label_names = np.apply_along_axis(get_label,1,train_labels)

test_label_names = np.apply_along_axis(get_label,1,test_labels)

valid_label_names = np.apply_along_axis(get_label,1,valid_labels)

In [ ]:
train_labels[:9]

In [12]:
from pickle import load

In [15]:
train_images_aug, train_labels_aug = load(open('train_aug.pickle','rb'))

In [16]:
train_images_aug.shape

(6000, 224, 224, 3)

In [17]:
train_labels_aug.shape

(6000, 4)

In [18]:
x_train = train_images.reshape(4098,150528)

x_test = test_images.reshape(1279,150528)

x_valid = valid_images.reshape(1023,150528)

In [19]:
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.optimizers import Adam, SGD
from keras.metrics import Recall, Precision 
from keras.callbacks import EarlyStopping

## Basic Convoluted Neural Network

In [ ]:
batch_size = 12
num_classes = 4
epochs = 30


y_train = keras.utils.to_categorical(train_labels, num_classes)

y_valid = keras.utils.to_categorical(valid_labels, num_classes)


cnn = models.Sequential()
cnn.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224,  3),use_bias=True))
cnn.add(layers.MaxPooling2D((2, 2)))
cnn.add(layers.Conv2D(32, (3, 3), activation='relu',use_bias=True))
cnn.add(layers.MaxPooling2D((2, 2)))
cnn.add(layers.Flatten())
cnn.add(layers.Dense(32, activation='relu'))
cnn.add(layers.Dense(4, activation='softmax'))

cnn.summary()

cnn.compile(loss='categorical_crossentropy',
              optimizer= Adam(),
              metrics=[Recall(),'accuracy'])

early_stop = EarlyStopping(monitor='val_loss',patience=5,verbose=1,mode='auto')

#dont forget to change
history = cnn.fit(train_images, train_labels,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    callbacks = [early_stop],
                    validation_data=(valid_images, valid_labels))
score = cnn.evaluate(test_images, test_labels, verbose=0)
print('test loss:', score[0])
print('Test recall:', score[1])
print('Test Accuracy', score[2])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.plot(figsize=(10,8))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

## Basic CNN with Augmented

In [ ]:
print(train_images_aug.shape)
print(train_labels_aug.shape)

In [ ]:
batch_size = 50
num_classes = 4
epochs = 30

cnn = models.Sequential()
cnn.add(layers.Conv2D(64, (5, 5), activation='relu', input_shape=(224, 224,  3),use_bias=True))
cnn.add(layers.MaxPooling2D((2, 2)))
cnn.add(layers.Conv2D(32, (3, 3), activation='relu',use_bias=True))
cnn.add(layers.MaxPooling2D((2, 2)))
cnn.add(layers.Flatten())
cnn.add(layers.Dense(32, activation='relu'))
cnn.add(layers.Dense(4, activation='softmax'))

cnn.summary()

cnn.compile(loss='categorical_crossentropy',
              optimizer= Adam(),
              metrics=[Recall(),'accuracy'])

early_stop = EarlyStopping(monitor='val_loss',patience=5,verbose=1,mode='auto')

#dont forget to change
cnn_aug_history = cnn.fit(train_images_aug, train_labels_aug,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    callbacks = [early_stop],
                    validation_data=(valid_images, valid_labels))
score = cnn.evaluate(test_images, test_labels, verbose=0)
print('test loss:', score[0])
print('Test recall:', score[1])
print('Test Accuracy', score[2])

In [ ]:
cnn_aug_history_df = pd.DataFrame(cnn_aug_history.history)
history_df.plot(figsize=(10,8))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

## AlexNet Model

In [ ]:
batch_size = 24
num_classes = 4
epochs = 30

alex = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(224,224,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4, activation='softmax')
])

alex.summary()

alex.compile(loss='categorical_crossentropy',
              optimizer= Adam(),
              metrics=[Recall(),'accuracy'])

early_stop = EarlyStopping(monitor='val_loss',patience=5,verbose=1,mode='auto')

alex_history = alex.fit(train_images, train_labels,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    callbacks = [early_stop],
                    validation_data=(valid_images, valid_labels))

score = alex.evaluate(test_images, test_labels, verbose=0)

print('test loss:', score[0])
print('Test recall:', score[1])
print('Test Accuracy', score[2])

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 54, 54, 96)        34944     
_________________________________________________________________
batch_normalization_18 (Batc (None, 54, 54, 96)        384       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 26, 26, 96)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 26, 26, 256)       614656    
_________________________________________________________________
batch_normalization_19 (Batc (None, 26, 26, 256)       1024      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 12, 12, 256)       0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 12, 12, 384)      

In [20]:
batch_size = 24
num_classes = 4
epochs = 30

alex = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(224,224,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4, activation='softmax')
])

alex.summary()

alex.compile(loss='categorical_crossentropy',
              optimizer= Adam(),
              metrics=[Recall(),'accuracy'])

early_stop = EarlyStopping(monitor='val_loss',patience=5,verbose=1,mode='auto')

alex_history = alex.fit(train_images_aug, train_labels_aug,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    callbacks = [early_stop],
                    validation_data=(valid_images, valid_labels))

score = alex.evaluate(test_images, test_labels, verbose=0)

print('test loss:', score[0])
print('Test recall:', score[1])
print('Test Accuracy', score[2])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 54, 54, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 54, 54, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 26, 26, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 26, 26, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 384)       8

In [ ]:
history_df = pd.DataFrame(alex_history.history)
history_df.plot(figsize=(10,8))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

In [48]:
batch_size = 50
num_classes = 4
epochs = 30
np.random.seed(7)

frontiers = models.Sequential()

frontiers.add(layers.Conv2D(32,(32,32),padding= 'same',activation = 'relu', input_shape=(224,224,3),use_bias=True))
frontiers.add(layers.BatchNormalization())
frontiers.add(layers.MaxPooling2D((16,16),strides=2))
frontiers.add(layers.Conv2D(32,(16,16),padding ='same',activation = 'relu',use_bias=True))
frontiers.add(layers.BatchNormalization())
frontiers.add(layers.AveragePooling2D((8,8),strides=2))
frontiers.add(layers.Conv2D(64,(8,8),padding = 'same',activation = 'relu',use_bias=True))
frontiers.add(layers.BatchNormalization())
frontiers.add(layers.AveragePooling2D((8,8)))
frontiers.add(layers.Flatten())
frontiers.add(Dense(1024,activation ='relu'))
frontiers.add(Dense(1024,activation ='relu'))
frontiers.add(Dense(4,activation = 'softmax'))

frontiers.summary()

frontiers.compile(loss='categorical_crossentropy',
              optimizer= Adam(),
              metrics=[Recall(),'accuracy'])

early_stop = EarlyStopping(monitor='val_loss',patience=5,verbose=1,mode='auto')

frontiers_history = frontiers.fit(train_images_aug, train_labels_aug,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    callbacks = [early_stop],
                    validation_data=(valid_images, valid_labels))

score = frontiers.evaluate(test_images, test_labels, verbose=0)

print('test loss:', score[0])
print('Test recall:', score[1])
print('Test Accuracy', score[2])


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 224, 224, 32)      98336     
_________________________________________________________________
batch_normalization_15 (Batc (None, 224, 224, 32)      128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 105, 105, 32)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 105, 105, 32)      262176    
_________________________________________________________________
batch_normalization_16 (Batc (None, 105, 105, 32)      128       
_________________________________________________________________
average_pooling2d_6 (Average (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 49, 49, 64)       

In [40]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.resnet50 import decode_predictions

In [22]:
model_res = keras.applications.resnet50.ResNet50(weights='imagenet')

102973440/102967424 [==============================] - 22s 0us/step


AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'reshape'

In [31]:
images_resized = tf.image.resize(train_images_aug, [224,224])

In [32]:
inputs = keras.applications.resnet50.preprocess_input(images_resized*255)

In [33]:
Y_proba = model_res.predict(inputs)

In [41]:
top_K = keras.applications.resnet50.decode_predictions(Y_proba, top=4)

ValueError: `decode_predictions` expects a batch of predictions (i.e. a 2D array of shape (samples, 1000)). Found array with shape: ()

In [37]:

for image_index in range(len(train_images_aug)):
    print('Image #{}'.format(image_index))
    for class_id, name, Y_proba in top_K[image_index]:
        print(' {} - {:12s}{:.2f}%.format'(class_id,name,Y_proba*100))
    print()


<>:5: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
<>:5: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?


ValueError: `decode_predictions` expects a batch of predictions (i.e. a 2D array of shape (samples, 1000)). Found array with shape: ()